### Importing libraries

In [1]:
import pandas as pd
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import string

#### train test files to one file

In [2]:
import os 


train_fake_dir = 'fake_news\\Train\\Fake'
train_real_dir = 'fake_news\\Train\\Real'
test_real_dir = 'fake_news\\Test\\Real'
test_fake_dir = 'fake_news\\Test\\Fake'

def getFilesPath(dir_path):
    f = []
    for root, dirs, filenames in os.walk(dir_path):
        for filename in filenames:
            f.append(os.path.join(root, filename))
    return f

train_fake_files = getFilesPath(train_fake_dir)
train_real_files = getFilesPath(train_real_dir)
test_fake_files = getFilesPath(test_fake_dir)
test_real_files = getFilesPath(test_real_dir)

print(
    len(train_fake_files),
    len(train_real_files),
    len(test_fake_files),
    len(test_real_files)
)

438 600 112 150


In [3]:
def getFilesContent(file_paths):
    content = []
    for file in file_paths:
        with open(file, 'r', encoding='utf-8') as f:
            content.append(f.read())
    return content


train_fake = getFilesContent(train_fake_files)
train_real = getFilesContent(train_real_files)
test_fake = getFilesContent(test_fake_files)
test_real = getFilesContent(test_real_files)

del train_fake_files
del train_real_files
del test_fake_files
del test_real_files

In [4]:
train_fake = pd.DataFrame({'text': train_fake})
train_real = pd.DataFrame({'text': train_real})
test_fake = pd.DataFrame({'text': test_fake})
test_real = pd.DataFrame({'text': test_real})

In [5]:
train_fake.head()

,text
0,\n\n\n عمران خان نے وزارتِ عظمیٰ کا عہدہ سنبھا...
1,پاکستان اسٹاک ایکسچینج (پی ایس ایکس) میں کاروب...
2,چین کی جانب سے پاکستان کی مدد کرنے سے انکار...
3,چین کی جانب سے پاکستان کی مدد کرنے سے انکار...
4,مرغی اور انڈوں کی قیمت میں مسلسل کمی کا اعلان...


In [6]:
train_fake['class'] = 0
train_real['class'] = 1
test_fake['class'] = 0
test_real['class'] = 1

In [7]:
train_fake.head()

,text,class
0,\n\n\n عمران خان نے وزارتِ عظمیٰ کا عہدہ سنبھا...,0
1,پاکستان اسٹاک ایکسچینج (پی ایس ایکس) میں کاروب...,0
2,چین کی جانب سے پاکستان کی مدد کرنے سے انکار...,0
3,چین کی جانب سے پاکستان کی مدد کرنے سے انکار...,0
4,مرغی اور انڈوں کی قیمت میں مسلسل کمی کا اعلان...,0


In [8]:
data = pd.concat([train_fake, train_real, test_fake, test_real], axis = 0)

In [9]:
data.columns

Index(['text', 'class'], dtype='object')

In [10]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [11]:
data.sample(frac = 1)

,text,class
21,اسٹیٹ بینک کا منافع چار گنا بڑھا\nنئی دہلی، 5 ...,1
173,ارجن من را مپال کال نے مہر حبیبہ سے دوبارہ شاد...,0
7,تصویر کے کاپی رائٹ Getty Images\n\nسعودی عرب ک...,1
497,ہوواوے پی 9 کے کیمرے کیسے کام کرتے ہیں؟\nہوواو...,1
235,کراچی (ڈیسک) ڈاکٹروں کی کمی پوری کر نے کے لئے ...,1
...,...,...
86,\nواشنگٹن (آن لائن) ٹرمپ انتظامیہ نے پاکستان ...,1
227,عالمی ادارۂ صحت کے مطابق دس لاکھ سے زائد افغا...,1
294,ورلڈکپ 2011 فکسنگ سکینڈل سامنے آگیا۔\nدبئی ۔ ک...,0
41,جرنل آف فیزیئولوجی میں شائع ہونے والی اس تحقیق...,1


In [12]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

#### Filter Text

In [13]:
data['text'] = data['text'].apply(wordopt)

#### Let's define independent and undependent variables

In [14]:
x = data['text']
y = data['class']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.1)

In [16]:
# convert text to vector

from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state=0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [18]:
pred_gb = GB.predict(xv_test)
GB.score(xv_test, y_test)

0.7076923076923077

In [22]:
def out_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test['text'].apply(wordopt)
    new_x_test = new_def_test['text']
    new_xv_test = vectorization.transform(new_x_test)
    # pred_GB = GB.predict(new_xv_test)
    # return print("\nzn GB Prediction: {}".format(out_lable(pred_GB[0])))
    return new_xv_test

In [20]:
news = str(input())
manual_testing(news)


zn GB Prediction: Not A Fake News


In [21]:
import pickle

with open('fake_news_det.pkl', 'wb') as f:
    pickle.dump(GB, f)


with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorization, f)

In [28]:
import pickle

with open('./fake_news_det.pkl', 'rb') as f:
    x = pickle.load(f)
    p = open('./vectorizer.pkl', 'rb')
    v = pickle.load(p)
    news = input()
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test['text'].apply(wordopt)
    new_x_test = new_def_test['text']
    new_xv_test = v.transform(new_x_test)
    # i = input()
    # t = manual_testing(i)
    r = x.predict(new_xv_test)
    print(r)


[1]
